In [1]:
%logstop
%logstart -rtq ~/.logs/DS_Classes_and_ORM.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# Class Example

Here we will build a few classes to represent some data, namely people in a company. The first class we need to define is a `Person` which is the general class representing a person. 

In [2]:
class Person(object):
    def __init__(self, name, age, degree, salary):
        self.name = name
        self.age = age
        self.degree = degree
        self.salary = salary
        
    def describe(self):
        fmt = "{} is {} years old, holds a {}, with a salary of {}"
        return fmt.format(self.name, self.age, self.degree, self.salary)

In [4]:
p = Person("Dejene", 30, "MSC in Computer Science", 300)
p.describe()

'Dejene is 30 years old, holds a MSC in Computer Science, with a salary of 300'

Now a degree is something we probably want to abstract as well, so lets create a degree class.

We might also do some validation, so we can use Mixin here.

In [5]:
class Validate(object):
    def _validate(self, ele, allowed):
        if ele not in allowed:
            raise ValueError('{} is not allowed for class {}'.format(ele, type(self)))

In [6]:
class Degree(Validate):
    ALLOWED_LEVELS = ['bachelors', 'masters', 'phd'] #class variable
    
    def __init__(self, level, area):
        self._validate(level, self.ALLOWED_LEVELS)
        self.level = level
        self.area = area
       
    def __repr__(self):
        return "{} in {}".format(self.level, self.area)

In [9]:
Degree('masters', 'computer science')

masters in computer science

In [11]:
class Salary(Validate):
    ALLOWED_PERIODS = {'weekly': 52.0,
                       'biweekly': 26.0,
                       'monthly': 12,
                       'quarterly': 4.0,
                       'annually':1.0}
    def __init__(self, amt, pay_period):
        self._validate(pay_period, self.ALLOWED_PERIODS)
        self.amt = amt
        self.period = pay_period
        
    def __repr__(self):
        return "${:.2f} per year".format(self.amt*self.ALLOWED_PERIODS[self.period])

In [13]:
Salary(300, 'monthly')

$3600.00 per year

In [14]:
p = Person('Eric', 25, Degree('masters', 'physics'), Salary(200, 'biweekly'))
p.describe()

'Eric is 25 years old, holds a masters in physics, with a salary of $5200.00 per year'

In [22]:
class Employee(Person):
    def __init__(self, idn, production, name, age, degree, salary):
        super(Employee, self).__init__(name, age, degree, salary)
        self.id = idn
        self.production = production
    
    def __repr__(self):
         return "ID: {}, name: {}".format(self.id, self.name)
        
class Manager(Employee):
    def __init__(self, capacity, idn, production, name, age, degree, salary):
        super(Manager, self).__init__(idn, production, name, age, degree, salary)
        self.capacity = capacity
        
    def can_manage(self, n):
        return n <= self.capacity

In [17]:
Manager(5, 181, 58200, 'Eric', 25, Degree('masters', 'physics'), Salary(200, 'biweekly')).can_manage(5)

True

In [21]:
class Group(object):
    def __init__(self):
        self.members = {}
    
    def add_member(self, member):
        self.members[member.id] = member
        
    def remove_member(self, member):
        del self.members[member.id]
        
    def get_member_ids(self):
        return self.members.keys()

In [23]:
class Team(Group):
    def __init__(self,idn, name, desc, manager=None):
        self.name = name
        self.id = idn
        self.desc = desc
        
        self.manager = None
        super(Team, self).__init__()
        
    def add_manager(self, manager):
        if self.manager is None:
            self.manager = manager.id
            self.add_member(manager)
        else:
            raise ValueError("can only have a single manager")
            
    def remove_manager(self):
        if self.manager is None:
            raise ValueError("cannot remove that which does not exist")
        else:
            del self.members[self.manager]
            self.manager = None
        
    def check_health(self):
        num_members = len(self.members)
        if num_members > 0 and self.manager is None:
            return False, "no manager for employees"
        elif self.manager is not None:
            if self.members[self.manager].can_manage(num_members - 1): # don't count self
                return True, ""
            else:
                return False, "too many employees for manager"
        else:
            return True, ""
                  
    def production(self):
        return sum([i.production for i in self.members.values()])
    
    def describe(self):
        return "team {} has {} members and produces ${}".format(self.name, len(self.members), self.production())
    
    def __repr__(self):
        return "ID: {}, Name: {}".format(self.id, self.name)

In [24]:
from collections import Counter
class Company(Group):
    def __init__(self, name):
        self.name = name
        super(Company, self).__init__()

    def production(self):
        return sum([i.production() for i in self.members.values()])
    
    def _single_assign(self):
        """check that each person is only on a single team"""
        ids = [j for i in self.members.values() for j in i.get_member_ids()]
        c = Counter(ids)
        return c.most_common(1)[0][1] == 1
    
    def check_health(self):
        problems = []
        if not self._single_assign():
            problems.append("employee assigned to more than one team")
        for k, v in self.members.items():
            health, reason = v.check_health()
            if not health:
                problem = "{} is not healthy because {}".format(v.name, reason)
                problems.append(problem)
        return problems
    
    def describe(self):
        problems = self.check_health()
        if not problems: #truthyness of lists
            print("{} is healthy and has production of ${}".format(self.name, self.production()))
            for k, team in self.members.items():
                print("  " + team.describe())

Often these can be mapped in a fairly straightforward way to a database.  We will describe a company here

In [25]:
# Salaries
low_salary = Salary(1800, 'biweekly')
mid_salary = Salary(5600, 'biweekly')
high_salary = Salary(100000, 'annually')

# Degrees
b_business = Degree('bachelors', 'business')
m_math = Degree('masters', 'mathematics')
b_finance = Degree('bachelors', 'finance')
b_physics = Degree('bachelors', 'physics')
p_math = Degree('phd', 'mathematics')

# Employees
fred = Employee(1, 120000, "Fred", 28, low_salary, b_business)
amy = Employee(2, 136000, "Amy", 29, low_salary, b_finance)
erica = Employee(5, 140000, "Erica", 47, mid_salary, b_physics)
jerry = Employee(7, 165000, "Jerry", 45, mid_salary, b_business)
sam = Employee(8, 127000, "Sam", 35, low_salary, b_finance)
danny = Employee(9, 128000, "Danny", 32, low_salary, m_math)
carrie = Employee(10, 120000, "Carrie", 39, low_salary, m_math)

# Managers
sally = Manager(5, 4, 131000, "Sally", 50, mid_salary, b_business)
arnold = Manager(10, 3, 125000, "Arnold", 60, mid_salary, m_math)
molly = Manager(10, 6, 180000, "Molly", 62, high_salary, p_math)

# Teams
tech = Team(1, 'tech', 'handles technical duties')
sales = Team(2, 'sales', 'handles sales duties')
executive = Team(3, 'executive', 'makes large business decisions')

for emp in [fred, amy, carrie]:
    tech.add_member(emp)

tech.add_manager(sally)

for emp in [erica, jerry, sam, danny]:
    sales.add_member(emp)

sales.add_manager(arnold)

executive.add_manager(molly)

# Company
company = Company("the-company")
for team in [tech, sales, executive]:
    company.add_member(team)

In [26]:
company.describe()

the-company is healthy and has production of $1372000
  team tech has 4 members and produces $507000
  team sales has 5 members and produces $685000
  team executive has 1 members and produces $180000


In [27]:
company.members

{1: ID: 1, Name: tech, 2: ID: 2, Name: sales, 3: ID: 3, Name: executive}

In [28]:
company.members[1].members[10]

ID: 10, name: Carrie

## Object-relation mapping (ORM)

There is a parallel between the interrelated classes we've defined and SQL tables. Just as we had a `Salary` class, a `Degree` class, an `Employee` class, etc. we could imagine a company having a `Salaries` table, `Degrees` table, and `Employees` table. Each row of one of these tables would be similar to a single object of that type (i.e. an instance of a class).

We can therefore imagine a _mapping_ between rows of a SQL table and Python objects. There are tools that make this mapping concrete, called object-relation mapping (ORM) tools. These tools can connect to databases and allow us to treat relations in the database as Python objects. This can be very useful for annotating SQL relations with global constants (such as an important date) and routine functions for analysis. It also allows for seamless integration of database relations into applications as fully-fledged Python objects.

We will use an ORM tool called `SQLAlchemy` to build a database from our example above, complete with not only the attributes of each object but also the reporting functions such as `check_health()`.

In [30]:
import sqlalchemy as sql # this is not conventional

We will follow the analogy of classes and tables; to create a table in our database, we declare a class. Our classes will inherit from a special base class that hooks into `SQLAlchemy`. **Be warned**, there's a lot going on in these class definitions and they will look different from other class definitions in order to support SQL operations. Looking below we'll notice several patterns:

1. All classes inherit from Base, possibly through other classes
1. All classes are associated with a SQL table via a `__tablename__` variable
1. There is no `__init__` method for initializing attributes; instead attributes are declared as variables of the type `sql.Column`
1. The attributes are typed; below we use the types `sql.Integer` and `sql.String`
1. All classes contain at least one `primary_key`
1. Relationships between classes/tables are accomplished through use of `sql.orm.relationship` and `sql.ForeignKey`

Some relationships managed by inheritance. Just as before, since `Manager` inherits from `Employee`, we don't need to add columns for production, name, salary, etc. At a SQL level, there is a `managers` table with `idn` and `capacity` columns that gets joined to the `employees` table (by `idn`) any time we want to know a manager's name (or production, or salary, etc.). This helps the SQL backend follow good database design principles.

In [31]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.hybrid import hybrid_property, hybrid_method

Base = declarative_base()

class Team(Base):
    __tablename__ = 'teams'

    idn = sql.Column(sql.Integer, primary_key=True)
    name = sql.Column(sql.String)

    members = sql.orm.relationship(
        'Employee',
        back_populates='team',
        cascade='all, delete-orphan')

    @hybrid_property
    def manager(self):
        for member in self.members:
            if isinstance(member, Manager):
                return member
        return None
    
    @hybrid_property
    def healthy(self):
        num_members = len(self.members)
        if len(self.members) > 0 and self.manager is None:
            return False
        elif self.manager is not None:
            if self.manager.can_manage(num_members - 1):
                return True
            else:
                return False
        else:
            return True

    @hybrid_property
    def production(self):
        return sum(member.production for member in self.members)

    def __repr__(self):
        return 'Team(idn={}, name={})'.format(self.idn, self.name)


class Employee(Base):
    __tablename__ = 'employees'

    idn = sql.Column(sql.Integer, primary_key=True)
    name = sql.Column(sql.String)
    age = sql.Column(sql.Integer)
    production = sql.Column(sql.Integer)
    team_idn = sql.Column(sql.ForeignKey('teams.idn'))
    salary_amt = sql.Column(sql.Integer)
    salary_period = sql.Column(sql.Integer)
    degree_level = sql.Column(sql.String)
    degree_area = sql.Column(sql.String)
    employee_type = sql.Column(sql.String)
    
    team = sql.orm.relationship('Team', back_populates='members')

    __mapper_args__ = {
        'polymorphic_identity': 'employee',
        'polymorphic_on': employee_type
    }

    @hybrid_property
    def salary(self):
        return self.salary_amt * self.salary_period

    def __repr__(self):
        return 'Employee(idn={}, name={}, age={}, production={}, team_idn={}, salary_amt={}, salary_period={}, degree_level={}, degree_area={}, employee_type={})'.format(self.idn, self.name, self.age, self.production, self.team_idn, self.salary_amt, self.salary_period, self.degree_level, self.degree_area, self.employee_type)


class Manager(Employee):
    __tablename__ = 'managers'
    
    idn = sql.Column(sql.ForeignKey('employees.idn'), primary_key=True)
    capacity = sql.Column(sql.Integer)

    __mapper_args__ = {
        'polymorphic_identity': 'manager',
    }

    @hybrid_method
    def can_manage(self, n):
        return n <= self.capacity

    def __repr__(self):
        return 'Employee(idn={}, age={}, production={}, team_idn={}, employee_type={})'.format(self.idn, self.age,
                                                                                                        self.production, self.team_idn,
                                                                                                        self.employee_type, self.capacity)

The `@hybrid_method` and `@hybrid_attribute` decorators are where `SQLAlchemy` starts to help us. Often we will be interested in some combination of attributes in a table. For instance, we might want to know an employee's annual salary. We can calculate this as their salary per pay period times the number of pay periods in the year. Rather than store the annual salary in the table (which would be redundant), we can tell `SQLAlchemy` to calculate this on the fly and treat it as an attribute. This way, I can interact with attributes of my objects in the same way whether they are actually stored in the underlying database or not. This can be particularly useful when the attribute has a complex definition.

Let's see `SQLAlchemy` in action. We'll create an `engine` that will manage database connections. The `echo=True` keyword will show us all the SQL that gets executed as we interact with `SQLAlchemy`.

In [32]:
engine = sql.create_engine('sqlite:///company.db', echo=True)

We will then prepare the database by creating all the tables that inherit from `Base`.

In [33]:
Base.metadata.create_all(engine)

2020-11-17 09:57:15,792 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-11-17 09:57:15,797 INFO sqlalchemy.engine.base.Engine ()
2020-11-17 09:57:15,799 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-11-17 09:57:15,801 INFO sqlalchemy.engine.base.Engine ()
2020-11-17 09:57:15,802 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("teams")
2020-11-17 09:57:15,807 INFO sqlalchemy.engine.base.Engine ()
2020-11-17 09:57:15,809 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("employees")
2020-11-17 09:57:15,809 INFO sqlalchemy.engine.base.Engine ()
2020-11-17 09:57:15,810 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("managers")
2020-11-17 09:57:15,811 INFO sqlalchemy.engine.base.Engine ()
2020-11-17 09:57:15,812 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE teams (
	idn INTEGER NOT NULL, 
	name VARCHAR, 
	PRIMARY KEY (idn)
)


2020-11-17 09:57:15,813 INFO sqlalchemy.eng

We will create a `Session` object which performs the object-relation mapping, i.e. the `Session` object is responsible for translating our actions with the objects we defined above into SQL, which can then be passed to the database through the `engine`.

In [34]:
session = sql.orm.Session(engine)

Finally, let's create our data and add it to the database.

In [35]:
# Salaries
periods = {'weekly': 52.0,
           'biweekly': 26.0,
           'monthly': 12,
           'quarterly': 4.0,
           'annually':1.0}

low_salary = dict(salary_amt=1800, salary_period=periods['biweekly'])
mid_salary = dict(salary_amt=5600, salary_period=periods['biweekly'])
high_salary = dict(salary_amt=100000, salary_period=periods['annually'])

# Degrees
b_business = dict(degree_level='bachelors', degree_area='business')
m_math = dict(degree_level='masters', degree_area='mathematics')
b_finance = dict(degree_level='bachelors', degree_area='finance')
b_physics = dict(degree_level='bachelors', degree_area='physics')
p_math = dict(degree_level='phd', degree_area='mathematics')

In [36]:
def cat_dict(*args):
    all_keys = [key for dictionary in args for key in dictionary.keys()]
    if len(set(all_keys)) < len(all_keys):
        raise ValueError("Keys are not unique!")
    return {k: v for dictionary in args for k, v in dictionary.items()}

In [37]:
# Employees
fred = Employee(name='Fred', production=120000, age=28, **cat_dict(low_salary, b_business))
amy = Employee(name='Amy', production=136000, age=29, **cat_dict(low_salary, b_finance))
erica = Employee(name='Erica', production=140000, age=47, **cat_dict(mid_salary, b_physics))
jerry = Employee(name='Jerry', production=165000, age=45, **cat_dict(mid_salary, b_business))
sam = Employee(name='Sam', production=127000, age=35, **cat_dict(low_salary, b_finance))
danny = Employee(name='Danny', production=128000, age=32, **cat_dict(low_salary, m_math))
carrie = Employee(name='Carrie', production=120000, age=39, **cat_dict(low_salary, m_math))

# Managers
sally = Manager(capacity=5, production=131000, name="Sally", age=50, **cat_dict(mid_salary, b_business))
arnold = Manager(capacity=10, production=125000, name="Arnold", age=60, **cat_dict(mid_salary, m_math))
molly = Manager(capacity=10, production=180000, name="Molly", age=62, **cat_dict(high_salary, p_math))

# Teams
tech = Team(name='tech', members=[fred, amy, carrie, sally])
sales = Team(name='sales', members=[erica, jerry, sam, danny, arnold])
executive = Team(name='executive', members=[molly])

# add the teams to the database; this also adds all attached objects
session.add(tech)
session.add(sales)
session.add(executive)

# commit the changes
session.commit()

2020-11-17 09:58:03,550 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-11-17 09:58:03,553 INFO sqlalchemy.engine.base.Engine INSERT INTO teams (name) VALUES (?)
2020-11-17 09:58:03,556 INFO sqlalchemy.engine.base.Engine ('tech',)
2020-11-17 09:58:03,559 INFO sqlalchemy.engine.base.Engine INSERT INTO teams (name) VALUES (?)
2020-11-17 09:58:03,560 INFO sqlalchemy.engine.base.Engine ('sales',)
2020-11-17 09:58:03,562 INFO sqlalchemy.engine.base.Engine INSERT INTO teams (name) VALUES (?)
2020-11-17 09:58:03,614 INFO sqlalchemy.engine.base.Engine ('executive',)
2020-11-17 09:58:03,618 INFO sqlalchemy.engine.base.Engine INSERT INTO employees (name, age, production, team_idn, salary_amt, salary_period, degree_level, degree_area, employee_type) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
2020-11-17 09:58:03,620 INFO sqlalchemy.engine.base.Engine ('Fred', 28, 120000, 1, 1800, 26.0, 'bachelors', 'business', 'employee')
2020-11-17 09:58:03,622 INFO sqlalchemy.engine.base.Engine INSERT INTO empl

In [38]:
# we can still interact with the objects we've created as Python objects...
tech.members

2020-11-17 09:58:05,534 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-11-17 09:58:05,536 INFO sqlalchemy.engine.base.Engine SELECT teams.idn AS teams_idn, teams.name AS teams_name 
FROM teams 
WHERE teams.idn = ?
2020-11-17 09:58:05,538 INFO sqlalchemy.engine.base.Engine (1,)
2020-11-17 09:58:05,543 INFO sqlalchemy.engine.base.Engine SELECT employees.idn AS employees_idn, employees.name AS employees_name, employees.age AS employees_age, employees.production AS employees_production, employees.team_idn AS employees_team_idn, employees.salary_amt AS employees_salary_amt, employees.salary_period AS employees_salary_period, employees.degree_level AS employees_degree_level, employees.degree_area AS employees_degree_area, employees.employee_type AS employees_employee_type 
FROM employees 
WHERE ? = employees.team_idn
2020-11-17 09:58:05,544 INFO sqlalchemy.engine.base.Engine (1,)
2020-11-17 09:58:05,547 INFO sqlalchemy.engine.base.Engine SELECT managers.capacity AS managers_capacit

[Employee(idn=1, name=Fred, age=28, production=120000, team_idn=1, salary_amt=1800, salary_period=26, degree_level=bachelors, degree_area=business, employee_type=employee),
 Employee(idn=2, name=Amy, age=29, production=136000, team_idn=1, salary_amt=1800, salary_period=26, degree_level=bachelors, degree_area=finance, employee_type=employee),
 Employee(idn=3, name=Carrie, age=39, production=120000, team_idn=1, salary_amt=1800, salary_period=26, degree_level=masters, degree_area=mathematics, employee_type=employee),
 Employee(idn=4, age=50, production=131000, team_idn=1, employee_type=manager)]

In [39]:
# or as SQL relations via the session
q = session.query(Team).filter(Team.name=='tech')

for team in q:
    for member in team.members:
        print(member)

2020-11-17 09:58:07,482 INFO sqlalchemy.engine.base.Engine SELECT teams.idn AS teams_idn, teams.name AS teams_name 
FROM teams 
WHERE teams.name = ?
2020-11-17 09:58:07,484 INFO sqlalchemy.engine.base.Engine ('tech',)
Employee(idn=1, name=Fred, age=28, production=120000, team_idn=1, salary_amt=1800, salary_period=26, degree_level=bachelors, degree_area=business, employee_type=employee)
Employee(idn=2, name=Amy, age=29, production=136000, team_idn=1, salary_amt=1800, salary_period=26, degree_level=bachelors, degree_area=finance, employee_type=employee)
Employee(idn=3, name=Carrie, age=39, production=120000, team_idn=1, salary_amt=1800, salary_period=26, degree_level=masters, degree_area=mathematics, employee_type=employee)
Employee(idn=4, age=50, production=131000, team_idn=1, employee_type=manager)


In [40]:
# a hybrid_attribute behaves just like a regular attribute
# even though it is not actually stored in the database or the object
print(tech.production)
print(tech.healthy)

507000
True


Using the `query` method of our session, we can perform all our usual SQL operations: filtering, joining, aggregating, etc.

In [41]:
a_employees = session.query(Employee).filter(Employee.name.like('A%'))
for a in a_employees:
    print(a)

2020-11-17 10:00:48,410 INFO sqlalchemy.engine.base.Engine SELECT employees.idn AS employees_idn, employees.name AS employees_name, employees.age AS employees_age, employees.production AS employees_production, employees.team_idn AS employees_team_idn, employees.salary_amt AS employees_salary_amt, employees.salary_period AS employees_salary_period, employees.degree_level AS employees_degree_level, employees.degree_area AS employees_degree_area, employees.employee_type AS employees_employee_type 
FROM employees 
WHERE employees.name LIKE ?
2020-11-17 10:00:48,411 INFO sqlalchemy.engine.base.Engine ('A%',)
Employee(idn=2, name=Amy, age=29, production=136000, team_idn=1, salary_amt=1800, salary_period=26, degree_level=bachelors, degree_area=finance, employee_type=employee)
2020-11-17 10:00:48,415 INFO sqlalchemy.engine.base.Engine SELECT managers.capacity AS managers_capacity 
FROM managers 
WHERE ? = managers.idn
2020-11-17 10:00:48,416 INFO sqlalchemy.engine.base.Engine (9,)
Employee(idn

In [42]:
for total in session.query(Team.name, sql.func.sum(Employee.salary)).join(Employee.team).group_by(Team.name):
    print(total)

2020-11-17 10:00:52,410 INFO sqlalchemy.engine.base.Engine SELECT teams.name AS teams_name, sum(employees.salary_amt * employees.salary_period) AS sum_1 
FROM employees JOIN teams ON teams.idn = employees.team_idn GROUP BY teams.name
2020-11-17 10:00:52,412 INFO sqlalchemy.engine.base.Engine ()
('executive', 100000)
('sales', 530400)
('tech', 286000)


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*